# Universidad Nacional de Colombia  
## Sede Bogotá  
### Facultad de Ingeniería
### Departamento de Ingeniería de Sistemas e Industrial

---

## **Inteligencia Artificial y Mini-robots**

**Título del Taller:**  
*Autómatas Celulares*

**Autor:**  
*Juan Jeronimo Gómez Rubiano*  

**Periodo:**  
*2025-2*

**Fecha de entrega:**  
*Septiembre 09/2025*

---

*El source code junto a este documento es encuentran disponible en el siguiente repositorio público de github: https://github.com/jujgomezru/ia-minirobots*


## Ejercicio 1

### Observe sus comportamientos en la casa, en la universidad y en el medio de transporte que utiliza. Encuentre, para cada uno de estos escenarios sus reglas básicas.

## Ejercicio 2

### Suponga una enfermedad, o un incendio forestal, o una moda, desarrolle un modelo de difusión usando ACs probabilísticos. O simule un robot con dos ruedas que evite obstáculos. Simule el comportamiento de un robot con tres sensores de distancia, que recorre un espacio bidimensional, donde hay 4 objetos distribuidos aleatoriamente, que no se choca con esos objetos.

Para este ejercicio, vamos a simular un robot con dos ruedas y tres sensores de distancia, que recorre el espacio bidimensional, con el objetivo de
esquivar los obstáculos y poder moverse a través del espacio, usando un Autómata Celular Probabilístico.

In [3]:
!pip install numpy
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 548.1 kB/s eta 0:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 7.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 2.0.0 requires scipy>=1.5, which is not installed.
qiskit 2.0.0 requires sympy>=1.3, which is not installed.
Defaulting to user installation because normal site-packages is not writeable
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 6.3 MB/s eta

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from IPython.display import display, clear_output
import time
import warnings
warnings.filterwarnings('ignore')

# Configuración de la simulación
WORLD_SIZE = 20
NUM_OBSTACLES = 4
SENSOR_RANGE = 16
DT = 0.1
PROBABILISTIC_BEHAVIOR = 0.2

# Velocidades aumentadas
SPEED_FORWARD = 0.8
SPEED_BACKWARD = 0.5

# Tabla de reglas
RULES_TABLE = {
    (0, 0, 0): (2, 2),
    (0, 0, 1): (2, 0),
    (0, 0, 2): (2, 0),
    (0, 1, 0): (0, 2),
    (0, 1, 1): (2, 0),
    (0, 1, 2): (2, 0),
    (0, 2, 0): (2, 0),
    (0, 2, 1): (2, 0),
    (0, 2, 2): (2, 0),
    (1, 0, 0): (2, 0),
    (1, 0, 1): (2, 0),
    (1, 0, 2): (2, 0),
    (1, 1, 0): (0, 2),
    (1, 1, 1): (1, 0),
    (1, 1, 2): (0, 2),
    (1, 2, 0): (0, 2),
    (1, 2, 1): (1, 0),
    (1, 2, 2): (0, 2),
    (2, 0, 0): (0, 2),
    (2, 0, 1): (0, 1),
    (2, 0, 2): (0, 1),
    (2, 1, 0): (1, 0),
    (2, 1, 1): (1, 0),
    (2, 1, 2): (1, 1),
    (2, 2, 0): (1, 0),
    (2, 2, 1): (1, 1),
    (2, 2, 2): (1, 1)
}

class Robot:
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
        self.sensors = {'I': 2, 'C': 2, 'D': 2}
        self.motors = {'Mi': 1, 'Md': 1}
        self.trail = []
        self.collisions = 0
        self.total_steps = 0
        self.border_collisions = 0
        
    def update_sensors(self, obstacles):
        # Reiniciar sensores
        self.sensors = {'I': 2, 'C': 2, 'D': 2}
        
        # Primero: detectar bordes del mundo
        self._detect_borders()
        
        # Segundo: detectar obstáculos (sobrescriben las detecciones de bordes si están más cerca)
        self._detect_obstacles(obstacles)
                
    def _detect_borders(self):
        # Detectar distancia a los bordes para cada sensor
        for sensor in ['I', 'C', 'D']:
            if sensor == 'C':
                sensor_angle = self.theta
            elif sensor == 'I':
                sensor_angle = self.theta + np.pi/4
            elif sensor == 'D':
                sensor_angle = self.theta - np.pi/4
                
            # Calcular distancia a los bordes en la dirección del sensor
            border_dist = self._distance_to_border(sensor_angle)
            
            # Actualizar estado del sensor si el borde está más cerca que obstáculos anteriores
            if border_dist < 16:  # Solo si está dentro del rango de detección
                if border_dist < 8 and self.sensors[sensor] > 0:
                    self.sensors[sensor] = 0
                elif border_dist < 16 and self.sensors[sensor] > 1:
                    self.sensors[sensor] = 1
    
    def _distance_to_border(self, angle):
        # Calcular distancia al borde más cercano en la dirección dada
        # Usamos geometría para calcular la intersección con los bordes
        
        # Coordenadas del sensor
        sensor_x = self.x + np.cos(angle) * 0.5
        sensor_y = self.y + np.sin(angle) * 0.5
        
        # Calcular distancia a cada borde
        dist_right = (WORLD_SIZE - sensor_x) / max(0.001, np.cos(angle)) if np.cos(angle) > 0 else SENSOR_RANGE
        dist_left = (-sensor_x) / max(0.001, -np.cos(angle)) if np.cos(angle) < 0 else SENSOR_RANGE
        dist_top = (WORLD_SIZE - sensor_y) / max(0.001, np.sin(angle)) if np.sin(angle) > 0 else SENSOR_RANGE
        dist_bottom = (-sensor_y) / max(0.001, -np.sin(angle)) if np.sin(angle) < 0 else SENSOR_RANGE
        
        # Tomar la distancia mínima positiva
        min_dist = min([d for d in [dist_right, dist_left, dist_top, dist_bottom] if d > 0])
        
        return min(min_dist, SENSOR_RANGE)
    
    def _detect_obstacles(self, obstacles):
        for sensor in self.sensors:
            if sensor == 'C':
                sensor_angle = self.theta
            elif sensor == 'I':
                sensor_angle = self.theta + np.pi/4
            elif sensor == 'D':
                sensor_angle = self.theta - np.pi/4
                
            sensor_x = self.x + np.cos(sensor_angle) * 0.5
            sensor_y = self.y + np.sin(sensor_angle) * 0.5
            
            min_dist = SENSOR_RANGE
            for obs in obstacles:
                if isinstance(obs, Circle):
                    dist = np.sqrt((sensor_x - obs.center[0])**2 + 
                                  (sensor_y - obs.center[1])**2) - obs.radius
                    if dist < min_dist:
                        min_dist = max(0, dist)
            
            # Actualizar estado del sensor si el obstáculo está más cerca que el borde
            if min_dist < 8:
                self.sensors[sensor] = 0
            elif min_dist < 16:
                self.sensors[sensor] = 1
                
    def update_motors(self):
        C, D, I = self.sensors['C'], self.sensors['D'], self.sensors['I']
        
        if np.random.random() < PROBABILISTIC_BEHAVIOR:
            self.motors['Mi'] = np.random.choice([0, 1, 2])
            self.motors['Md'] = np.random.choice([0, 1, 2])
        else:
            self.motors['Mi'], self.motors['Md'] = RULES_TABLE[(C, D, I)]
            
    def check_collision(self, obstacles):
        # Verificar colisión con bordes
        if self.x <= 0.5 or self.x >= WORLD_SIZE - 0.5 or self.y <= 0.5 or self.y >= WORLD_SIZE - 0.5:
            self.collisions += 1
            self.border_collisions += 1
            return True
            
        # Verificar colisión con obstáculos
        for obs in obstacles:
            if isinstance(obs, Circle):
                dist = np.sqrt((self.x - obs.center[0])**2 + 
                              (self.y - obs.center[1])**2)
                if dist < (0.5 + obs.radius):
                    self.collisions += 1
                    return True
        return False
            
    def update_position(self):
        v_left = SPEED_FORWARD if self.motors['Mi'] == 1 else (-SPEED_BACKWARD if self.motors['Mi'] == 2 else 0)
        v_right = SPEED_FORWARD if self.motors['Md'] == 1 else (-SPEED_BACKWARD if self.motors['Md'] == 2 else 0)
        
        v_linear = (v_left + v_right) / 2
        v_angular = (v_right - v_left) / 0.5
        
        self.theta += v_angular * DT
        new_x = self.x + v_linear * np.cos(self.theta) * DT
        new_y = self.y + v_linear * np.sin(self.theta) * DT
        
        prev_x, prev_y = self.x, self.y
        self.x = new_x
        self.y = new_y
        
        if self.check_collision(OBSTACLES):
            self.x, self.y = prev_x, prev_y
            self.theta += np.random.uniform(-np.pi/2, np.pi/2)
        
        # Forzar a mantenerse dentro de los límites
        self.x = np.clip(self.x, 0.5, WORLD_SIZE-0.5)
        self.y = np.clip(self.y, 0.5, WORLD_SIZE-0.5)
        
        self.trail.append((self.x, self.y))
        self.total_steps += 1

# Generar obstáculos aleatorios
OBSTACLES = []
for _ in range(NUM_OBSTACLES):
    x = np.random.uniform(2, WORLD_SIZE-2)
    y = np.random.uniform(2, WORLD_SIZE-2)
    OBSTACLES.append(Circle((x, y), radius=1))

# Crear robot en posición aleatoria (pero no demasiado cerca de los bordes)
robot = Robot(x=np.random.uniform(3, WORLD_SIZE-3), 
              y=np.random.uniform(3, WORLD_SIZE-3), 
              theta=np.random.uniform(0, 2*np.pi))

# Configurar la figura
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, WORLD_SIZE)
ax.set_ylim(0, WORLD_SIZE)
ax.set_aspect('equal')

# Dibujar borde del mundo como un rectángulo
border = Rectangle((0, 0), WORLD_SIZE, WORLD_SIZE, fill=False, color='black', linewidth=2)
ax.add_patch(border)

# Función para dibujar el estado actual
def draw_state():
    ax.clear()
    ax.set_xlim(0, WORLD_SIZE)
    ax.set_ylim(0, WORLD_SIZE)
    ax.set_aspect('equal')
    ax.set_title('Robot con AC - Bordes como Obstáculos')
    
    # Dibujar borde del mundo
    ax.add_patch(Rectangle((0, 0), WORLD_SIZE, WORLD_SIZE, fill=False, color='black', linewidth=2))
    
    # Dibujar obstáculos
    for obs in OBSTACLES:
        ax.add_patch(Circle((obs.center[0], obs.center[1]), radius=obs.radius, color='gray'))
    
    # Dibujar robot
    ax.add_patch(Circle((robot.x, robot.y), radius=0.5, fill=False, color='blue', linewidth=2))
    
    # Dibujar sensores
    colors = ['red', 'green', 'blue']
    for i, sensor in enumerate(['I', 'C', 'D']):
        if sensor == 'C':
            angle = robot.theta
        elif sensor == 'I':
            angle = robot.theta + np.pi/4
        elif sensor == 'D':
            angle = robot.theta - np.pi/4
        ax.plot([robot.x, robot.x + np.cos(angle) * 2], 
                [robot.y, robot.y + np.sin(angle) * 2], 
                color=colors[i], linewidth=1.5, alpha=0.7)
    
    # Dibujar trayectoria
    if len(robot.trail) > 1:
        x_trail, y_trail = zip(*robot.trail)
        ax.plot(x_trail, y_trail, 'g-', alpha=0.6, linewidth=2)
    
    # Añadir texto de estado
    state_str = f'Sensores: I={robot.sensors["I"]}, C={robot.sensors["C"]}, D={robot.sensors["D"]}\n'
    state_str += f'Motores: Mi={robot.motors["Mi"]}, Md={robot.motors["Md"]}\n'
    state_str += f'Posición: ({robot.x:.2f}, {robot.y:.2f})'
    ax.text(0.02, 0.95, state_str, transform=ax.transAxes, fontsize=9, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    # Añadir texto de colisiones
    collision_str = f'Colisiones totales: {robot.collisions}\n'
    collision_str += f'Colisiones bordes: {robot.border_collisions}\n'
    collision_str += f'Pasos: {robot.total_steps}\n'
    collision_str += f'Tasa colisión: {robot.collisions/max(1, robot.total_steps)*100:.1f}%'
    ax.text(0.02, 0.85, collision_str, transform=ax.transAxes, fontsize=9,
            bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8))
    
    # Mostrar la figura
    display(fig)
    clear_output(wait=True)

# Simulación continua
print("Simulación iniciada. Los bordes son tratados como obstáculos.")
print("Presiona el botón de stop en Jupyter para detener.")
try:
    while True:
        # Actualizar el robot
        robot.update_sensors(OBSTACLES)
        robot.update_motors()
        robot.update_position()
        
        # Dibujar el estado actual
        draw_state()
        
        # Pequeña pausa
        time.sleep(0.1)
        
        # Limitar la longitud del trail para mejor rendimiento
        if len(robot.trail) > 200:
            robot.trail = robot.trail[-200:]
            
except KeyboardInterrupt:
    print("\nSimulación detenida por el usuario.")
finally:
    plt.close(fig)
    print(f"Simulación completada. Total de pasos: {robot.total_steps}")
    print(f"Colisiones totales: {robot.collisions}")
    print(f"Colisiones con bordes: {robot.border_collisions}")


Simulación detenida por el usuario.
Simulación completada. Total de pasos: 300
Colisiones totales: 15
Colisiones con bordes: 8


## Ejercicio 3

### Tome el plano de una ciudad pequeña y localice, por ejemplo, las droguerías, centros de atención de salud y colegios. Por cada concepto dibuje un diagrama de Voronoi. ¿Considera que puede faltar una droguería, o un centro de atención de salud o un colegio? ¿Hay alguna relación entre los diagramas?

In [1]:
# Primero, instalamos las dependencias necesarias (ejecutar solo una vez)
!pip install opencv-python scipy matplotlib numpy

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 1.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 1.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 1.6 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.0 MB/s eta 0:00:0000:0100:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
Using 

In [ ]:
# Importamos las bibliotecas necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
import os
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Función para detectar puntos de interés y generar diagrama de Voronoi
def detectar_puntos_interes(image_path, color_range):
    """
    Detecta puntos de interés en un mapa basándose en su color y genera un diagrama de Voronoi.
    
    Args:
        image_path: Ruta a la imagen del mapa
        color_range: Rango de color en HSV para detectar los puntos de interés ([lower_h, lower_s, lower_v], [upper_h, upper_s, upper_v])
    """
    # Cargar la imagen
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: No se pudo cargar la imagen {image_path}")
        return
    
    # Convertir a espacio de color HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Definir rango de color para detectar
    lower_color = np.array(color_range[0])
    upper_color = np.array(color_range[1])
    
    # Crear máscara para el color especificado
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Operaciones morfológicas para mejorar la detección
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Encontrar contornos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Extraer coordenadas de los puntos de interés
    puntos = []
    for contour in contours:
        # Calcular el centro del contorno
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            puntos.append((cX, cY))
    
    if not puntos:
        print("No se detectaron puntos de interés con el color especificado.")
        return
    
    # Crear diagrama de Voronoi
    vor = Voronoi(puntos)
    
    # Visualizar resultados
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Imagen original con puntos detectados
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.scatter(*zip(*puntos), c='red', s=30)
    ax1.set_title('Mapa con puntos de interés detectados')
    ax1.axis('off')
    
    # Diagrama de Voronoi
    voronoi_plot_2d(vor, ax=ax2, show_vertices=False, line_colors='orange', line_width=2, line_alpha=0.6, point_size=10)
    ax2.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax2.set_title('Diagrama de Voronoi para puntos de interés')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return puntos, vor

# Función para listar las imágenes disponibles en el directorio
def listar_imagenes():
    """Lista todas las imágenes PNG y JPG en el directorio actual"""
    imagenes = []
    for ext in ['*.png', '*.jpg', '*.jpeg']:
        imagenes.extend([f for f in os.listdir('.') if f.lower().endswith(ext[1:])])
    return imagenes

# Interfaz interactiva para Jupyter Notebook
def interfaz_voronoi():
    """Crea una interfaz interactiva para seleccionar imágenes y ajustar parámetros"""
    
    # Listar imágenes disponibles
    imagenes = listar_imagenes()
    
    if not imagenes:
        print("No se encontraron imágenes en el directorio actual.")
        return
    
    # Crear widgets para la interfaz
    imagen_widget = widgets.Dropdown(
        options=imagenes,
        value=imagenes[0],
        description='Imagen:',
        disabled=False,
    )
    
    # Widgets para ajustar el rango de color HSV
    h_low = widgets.IntSlider(value=0, min=0, max=179, description='H Low')
    s_low = widgets.IntSlider(value=100, min=0, max=255, description='S Low')
    v_low = widgets.IntSlider(value=100, min=0, max=255, description='V Low')
    h_high = widgets.IntSlider(value=10, min=0, max=179, description='H High')
    s_high = widgets.IntSlider(value=255, min=0, max=255, description='S High')
    v_high = widgets.IntSlider(value=255, min=0, max=255, description='V High')
    
    # Botón para ejecutar
    ejecutar_btn = widgets.Button(description="Generar Diagrama")
    
    # Output para mostrar resultados
    out = widgets.Output()
    
    # Función para manejar el clic del botón
    def on_ejecutar_clicked(b):
        with out:
            clear_output(wait=True)
            color_range = [
                [h_low.value, s_low.value, v_low.value],
                [h_high.value, s_high.value, v_high.value]
            ]
            print(f"Procesando {imagen_widget.value} con rango de color {color_range}...")
            detectar_puntos_interes(imagen_widget.value, color_range)
    
    # Asignar la función al botón
    ejecutar_btn.on_click(on_ejecutar_clicked)
    
    # Mostrar la interfaz
    display(widgets.VBox([
        imagen_widget,
        widgets.HBox([h_low, h_high]),
        widgets.HBox([s_low, s_high]),
        widgets.HBox([v_low, v_high]),
        ejecutar_btn,
        out
    ]))

# Ejecutar la interfaz
interfaz_voronoi()